This notebook demonstrates a post-processing strategy for the
[Indoor Location & Navigation](https://www.kaggle.com/c/indoor-location-navigation)
competition.

To combine machine learning (wifi features) predictions with sensor data (acceleration, attitude heading),
I defined cost function as follows,
$$
L(X_{1:N}) = \sum_{i=1}^{N} \alpha_i \| X_i - \hat{X}_i \|^2 + \sum_{i=1}^{N-1} \beta_i \| (X_{i+1} - X_{i}) - \Delta \hat{X}_i \|^2
$$
where $\hat{X}_i$ is absolute position predicted by machine learning and $\Delta \hat{X}_i$ is relative position predicted by sensor data.

Since the cost function is quadratic, the optimal $X$ is solved by linear equation $Q X = c$
, where $Q$ and $c$ are derived from above cost function.
Because the matrix $Q$ is tridiagonal,
each machine learning prediction is corrected by *all* machine learning predictions and sensor data.

The optimal hyperparameters ($\alpha$ and $\beta$) can be estimated by expected error of machine learning and sensor data,
or just tuned by public score.

## References
+ [Simple 99% Accurate Floor Model](https://www.kaggle.com/nigelhenry/simple-99-accurate-floor-model)
+ [Indoor Location Competition 2.0 (Sample Data and Code)](https://github.com/location-competition/indoor-location-competition-20)

In [1]:
import sys
sys.path.append('/home/knikaido/work/Indoor-Location-Navigation/Git/indoor_location_competition_20')

In [2]:
import multiprocessing
import numpy as np
import pandas as pd
import scipy.interpolate
import scipy.sparse
from tqdm import tqdm
from pathlib import Path

from io_f import read_data_file
import compute_f as compute_f

In [3]:
DATA_DIR = Path("/home/knikaido/work/Indoor-Location-Navigation/data/")
TXT_DIR = DATA_DIR / 'indoor-location-navigation'
INPUT_PATH = str(TXT_DIR)
WIFI_DIR = DATA_DIR / 'indoorunifiedwifids_original'


In [4]:
def compute_rel_positions(acce_datas, ahrs_datas):
    step_timestamps, step_indexs, step_acce_max_mins = compute_f.compute_steps(acce_datas)
    headings = compute_f.compute_headings(ahrs_datas)
    stride_lengths = compute_f.compute_stride_length(step_acce_max_mins)
    step_headings = compute_f.compute_step_heading(step_timestamps, headings)
    rel_positions = compute_f.compute_rel_positions(stride_lengths, step_headings)
    return rel_positions

In [5]:
def correct_path(args):
    path, path_df = args
    
    T_ref  = path_df['timestamp'].values
    xy_hat = path_df[['x', 'y']].values
    
    example = read_data_file(f'{INPUT_PATH}/test/{path}.txt')
    rel_positions = compute_rel_positions(example.acce, example.ahrs)
    if T_ref[-1] > rel_positions[-1, 0]:
        rel_positions = [np.array([[0, 0, 0]]), rel_positions, np.array([[T_ref[-1], 0, 0]])]
    else:
        rel_positions = [np.array([[0, 0, 0]]), rel_positions]
    rel_positions = np.concatenate(rel_positions)
    
    T_rel = rel_positions[:, 0]
    delta_xy_hat = np.diff(scipy.interpolate.interp1d(T_rel, np.cumsum(rel_positions[:, 1:3], axis=0), axis=0)(T_ref), axis=0)

    N = xy_hat.shape[0]
    delta_t = np.diff(T_ref)
    alpha = (8.1)**(-2) * np.ones(N)
    beta  = (0.3 + 0.3 * 1e-3 * delta_t)**(-2)
    A = scipy.sparse.spdiags(alpha, [0], N, N)
    B = scipy.sparse.spdiags( beta, [0], N-1, N-1)
    D = scipy.sparse.spdiags(np.stack([-np.ones(N), np.ones(N)]), [0, 1], N-1, N)

    Q = A + (D.T @ B @ D)
    c = (A @ xy_hat) + (D.T @ (B @ delta_xy_hat))
    xy_star = scipy.sparse.linalg.spsolve(Q, c)

    return pd.DataFrame({
        'site_path_timestamp' : path_df['site_path_timestamp'],
        'floor' : path_df['floor'],
        'x' : xy_star[:, 0],
        'y' : xy_star[:, 1],
    })

In [6]:
def calc_delta(args):
    path, path_df = args
    zeros = np.zeros([1, 2], dtype=np.float32)
    
    T_ref  = path_df['timestamp'].values   
    txt_path = path_df['txt_path'].values[0]

    example = read_data_file(txt_path)
    rel_positions = compute_rel_positions(example.acce, example.ahrs)
    if T_ref[-1] > rel_positions[-1, 0]:
        rel_positions = [np.array([[0, 0, 0]]), rel_positions, np.array([[T_ref[-1], 0, 0]])]
    else:
        rel_positions = [np.array([[0, 0, 0]]), rel_positions]
    rel_positions = np.concatenate(rel_positions)
    
    T_rel = rel_positions[:, 0]
    delta_xy_hat = np.diff(scipy.interpolate.interp1d(T_rel, np.cumsum(rel_positions[:, 1:3], axis=0), axis=0)(T_ref), axis=0)
    delta_xy_hat = np.vstack([zeros, delta_xy_hat])
    
#     return np.conca delta_xy_hat
    return pd.DataFrame({
        'timestamp' : path_df['timestamp'],
        'delta_x' : delta_xy_hat[:, 0],
        'delta_y' : delta_xy_hat[:, 1],
    })

In [7]:
import glob
sub_df = pd.read_csv(DATA_DIR/'indoor-location-navigation/sample_submission.csv')
tmp = sub_df['site_path_timestamp'].apply(lambda s : pd.Series(s.split('_')))
sub_df['site'] = tmp[0]
sub_df['path'] = tmp[1]
sub_df['timestamp'] = tmp[2].astype(float)
used_buildings = sorted(sub_df['site'].value_counts().index.tolist())
test_txts = sorted(glob.glob(str(DATA_DIR/'indoor-location-navigation') + f'/test/*.txt'))
train_txts = [sorted(glob.glob(str(DATA_DIR/'indoor-location-navigation') + f'/train/{used_building}/*/*.txt')) for used_building in used_buildings]
train_txts = sum(train_txts, [])

In [8]:
train_df = pd.read_csv(WIFI_DIR / 'train_10_th10000_base25_withpassedtime.csv')

In [9]:
train_df = train_df[['path_id', 'wp_tmestamp', 'site_id']]
train_df = train_df.rename(columns={'path_id':'path', 'wp_tmestamp':'timestamp', 'site_id':'site'})
train_df

,path,timestamp,site
0,5e158ef61506f2000638fd1f,1578469851129,5a0546857ecc773753327266
1,5e158ef61506f2000638fd1f,1578469857653,5a0546857ecc773753327266
2,5e158ef61506f2000638fd1f,1578469857653,5a0546857ecc773753327266
3,5e158ef61506f2000638fd1f,1578469857653,5a0546857ecc773753327266
4,5e158ef61506f2000638fd1f,1578469862177,5a0546857ecc773753327266
...,...,...,...
251108,5dcd5c9323759900063d590a,1573733061352,5dc8cea7659e181adb076a3f
251109,5dcd5c9323759900063d590a,1573733070079,5dc8cea7659e181adb076a3f
251110,5dcd5c9323759900063d590a,1573733070079,5dc8cea7659e181adb076a3f
251111,5dcd5c9323759900063d590a,1573733070079,5dc8cea7659e181adb076a3f


In [10]:
train_df = train_df[~train_df.duplicated(subset='timestamp')].reset_index()

In [11]:
txt_pathes = []
for path in tqdm(train_df['path'].values):
    txt_pathes.append([train_txt for train_txt in train_txts if path in train_txt][0])
train_df['txt_path'] = txt_pathes

100%|██████████| 70732/70732 [00:40<00:00, 1741.75it/s]


In [12]:
processes = multiprocessing.cpu_count()
with multiprocessing.Pool(processes=processes) as pool:
    dfs = pool.imap_unordered(calc_delta, train_df.groupby('path'))
    dfs = tqdm(dfs)
    dfs = list(dfs)

10789it [07:44, 23.24it/s]


In [13]:
train_delta = pd.concat(dfs).sort_index()
train_delta

,timestamp,delta_x,delta_y
0,1578469851129,0.000000,0.000000
1,1578469857653,3.277117,3.589049
2,1578469862177,3.677518,2.843589
3,1578468786260,0.000000,0.000000
4,1578468794097,4.967612,5.556134
...,...,...,...
70727,1573733041658,12.615541,1.868515
70728,1573733049941,15.520237,-0.944616
70729,1573733058003,13.278622,-1.678478
70730,1573733061352,1.020265,2.597020


In [14]:
train_delta = train_delta.rename(columns={ 'timestamp':'wp_tmestamp'})


In [15]:
train_df = pd.read_csv(WIFI_DIR / 'train_10_th10000_base25_withpassedtime.csv')

In [16]:
train_df = train_df.merge(train_delta, how='left', on='wp_tmestamp')

In [17]:
train_df.to_csv(DATA_DIR / 'indoorunifiedwifids_original/train_10_th10000_base25_withpassedtime_withdelta.csv', index=False)

In [18]:
test_df = pd.read_csv(WIFI_DIR / 'test_10_th10000_base25_withpassedtime.csv')
test_df

,ssid_0,ssid_1,ssid_2,ssid_3,ssid_4,ssid_5,ssid_6,ssid_7,ssid_8,ssid_9,...,frequency_95,frequency_96,frequency_97,frequency_98,frequency_99,passed_time,wp_tmestamp,path_id,site_id,site_path_timestamp
0,b9f0208be00bd8b337be7f12e02e3a3ce846e22b,ab150ecf6d972b476aeab16317bed6189d9f7cce,b6ffe5619e02871fcd04f61c9bb4b5c53a3f46b7,b7e6027447eb1f81327d66cfd3adbe557aabf26c,d839a45ebe64ab48b60a407d837fb01d3c0dfef9,7182afc4e5c212133d5d7d76eb3df6c24618302b,da39a3ee5e6b4b0d3255bfef95601890afd80709,01e78c97f2c04cdbc7c4159158bb125a9bb558ff,b9f0208be00bd8b337be7f12e02e3a3ce846e22b,7182afc4e5c212133d5d7d76eb3df6c24618302b,...,5785,5785,5785,5180,2417,2340,1578474563646,046cfa46be49fc10834815c6,5a0546857ecc773753327266,5a0546857ecc773753327266_046cfa46be49fc1083481...
1,da39a3ee5e6b4b0d3255bfef95601890afd80709,d839a45ebe64ab48b60a407d837fb01d3c0dfef9,b6ffe5619e02871fcd04f61c9bb4b5c53a3f46b7,b7e6027447eb1f81327d66cfd3adbe557aabf26c,d839a45ebe64ab48b60a407d837fb01d3c0dfef9,7182afc4e5c212133d5d7d76eb3df6c24618302b,b9f0208be00bd8b337be7f12e02e3a3ce846e22b,7182afc4e5c212133d5d7d76eb3df6c24618302b,b7e6027447eb1f81327d66cfd3adbe557aabf26c,b6ffe5619e02871fcd04f61c9bb4b5c53a3f46b7,...,5180,5180,5180,5180,2452,9508,1578474572654,046cfa46be49fc10834815c6,5a0546857ecc773753327266,5a0546857ecc773753327266_046cfa46be49fc1083481...
2,b6ffe5619e02871fcd04f61c9bb4b5c53a3f46b7,da39a3ee5e6b4b0d3255bfef95601890afd80709,b7e6027447eb1f81327d66cfd3adbe557aabf26c,d839a45ebe64ab48b60a407d837fb01d3c0dfef9,7182afc4e5c212133d5d7d76eb3df6c24618302b,b9f0208be00bd8b337be7f12e02e3a3ce846e22b,da39a3ee5e6b4b0d3255bfef95601890afd80709,b9f0208be00bd8b337be7f12e02e3a3ce846e22b,b6ffe5619e02871fcd04f61c9bb4b5c53a3f46b7,d839a45ebe64ab48b60a407d837fb01d3c0dfef9,...,2452,2457,5745,2437,2437,14714,1578474578963,046cfa46be49fc10834815c6,5a0546857ecc773753327266,5a0546857ecc773753327266_046cfa46be49fc1083481...
3,da39a3ee5e6b4b0d3255bfef95601890afd80709,da39a3ee5e6b4b0d3255bfef95601890afd80709,b9f0208be00bd8b337be7f12e02e3a3ce846e22b,d839a45ebe64ab48b60a407d837fb01d3c0dfef9,b6ffe5619e02871fcd04f61c9bb4b5c53a3f46b7,b7e6027447eb1f81327d66cfd3adbe557aabf26c,b7e6027447eb1f81327d66cfd3adbe557aabf26c,7182afc4e5c212133d5d7d76eb3df6c24618302b,5731b8e08abc69d4c4d685c58164059207c93310,d839a45ebe64ab48b60a407d837fb01d3c0dfef9,...,5240,5260,2432,2432,5240,19587,1578474582400,046cfa46be49fc10834815c6,5a0546857ecc773753327266,5a0546857ecc773753327266_046cfa46be49fc1083481...
4,da39a3ee5e6b4b0d3255bfef95601890afd80709,da39a3ee5e6b4b0d3255bfef95601890afd80709,7182afc4e5c212133d5d7d76eb3df6c24618302b,b9f0208be00bd8b337be7f12e02e3a3ce846e22b,b6ffe5619e02871fcd04f61c9bb4b5c53a3f46b7,b7e6027447eb1f81327d66cfd3adbe557aabf26c,26571eb257b6a34b6beb93c05768e751d19279a6,d839a45ebe64ab48b60a407d837fb01d3c0dfef9,d839a45ebe64ab48b60a407d837fb01d3c0dfef9,da39a3ee5e6b4b0d3255bfef95601890afd80709,...,5300,5300,5300,5260,5260,22074,1578474585965,046cfa46be49fc10834815c6,5a0546857ecc773753327266,5a0546857ecc773753327266_046cfa46be49fc1083481...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10101,0f927dce74ec3475c7a39299e5bffab222ca665d,5d998a8668536c4f51004c25f474117fe9555f78,8e69018f6343506344ab13646ccd9447fc2ffb19,efa1fe97c4b2a6c57d98de9c236a109032ceb839,da39a3ee5e6b4b0d3255bfef95601890afd80709,5d998a8668536c4f51004c25f474117fe9555f78,c72257cf75d6a411225948bbc4e37ad554e7beda,da39a3ee5e6b4b0d3255bfef95601890afd80709,5d998a8668536c4f51004c25f474117fe9555f78,da39a3ee5e6b4b0d3255bfef95601890afd80709,...,5805,5200,5200,5280,5785,81664,1573731143256,fd64de8c4a2fc5ebb0e9f412,5dc8cea7659e181adb076a3f,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...
10102,8e69018f6343506344ab13646ccd9447fc2ffb19,0f927dce74ec3475c7a39299e5bffab222ca665d,efa1fe97c4b2a6c57d98de9c236a109032ceb839,da39a3ee5e6b4b0d3255bfef95601890afd80709,5d998a8668536c4f51004c25f474117fe9555f78,5d998a8668536c4f51004c25f474117fe9555f78,da39a3ee5e6b4b0d3255bfef95601890afd80709,8e69018f6343506344ab13646ccd9447fc2ffb19,5d998a8668

In [19]:
base_path = DATA_DIR/'indoor-location-navigation'
ssubm_df_diff = pd.read_csv(base_path / 'test_path_timediff.csv')
test_df = test_df.merge(ssubm_df_diff, how='left', on='path_id')
test_df['wp_tmestamp'] = test_df['wp_tmestamp'].astype(int) - test_df['time_diff'].astype(int)
test_df

,ssid_0,ssid_1,ssid_2,ssid_3,ssid_4,ssid_5,ssid_6,ssid_7,ssid_8,ssid_9,...,frequency_96,frequency_97,frequency_98,frequency_99,passed_time,wp_tmestamp,path_id,site_id,site_path_timestamp,time_diff
0,b9f0208be00bd8b337be7f12e02e3a3ce846e22b,ab150ecf6d972b476aeab16317bed6189d9f7cce,b6ffe5619e02871fcd04f61c9bb4b5c53a3f46b7,b7e6027447eb1f81327d66cfd3adbe557aabf26c,d839a45ebe64ab48b60a407d837fb01d3c0dfef9,7182afc4e5c212133d5d7d76eb3df6c24618302b,da39a3ee5e6b4b0d3255bfef95601890afd80709,01e78c97f2c04cdbc7c4159158bb125a9bb558ff,b9f0208be00bd8b337be7f12e02e3a3ce846e22b,7182afc4e5c212133d5d7d76eb3df6c24618302b,...,5785,5785,5180,2417,2340,9,046cfa46be49fc10834815c6,5a0546857ecc773753327266,5a0546857ecc773753327266_046cfa46be49fc1083481...,1578474563637
1,da39a3ee5e6b4b0d3255bfef95601890afd80709,d839a45ebe64ab48b60a407d837fb01d3c0dfef9,b6ffe5619e02871fcd04f61c9bb4b5c53a3f46b7,b7e6027447eb1f81327d66cfd3adbe557aabf26c,d839a45ebe64ab48b60a407d837fb01d3c0dfef9,7182afc4e5c212133d5d7d76eb3df6c24618302b,b9f0208be00bd8b337be7f12e02e3a3ce846e22b,7182afc4e5c212133d5d7d76eb3df6c24618302b,b7e6027447eb1f81327d66cfd3adbe557aabf26c,b6ffe5619e02871fcd04f61c9bb4b5c53a3f46b7,...,5180,5180,5180,2452,9508,9017,046cfa46be49fc10834815c6,5a0546857ecc773753327266,5a0546857ecc773753327266_046cfa46be49fc1083481...,1578474563637
2,b6ffe5619e02871fcd04f61c9bb4b5c53a3f46b7,da39a3ee5e6b4b0d3255bfef95601890afd80709,b7e6027447eb1f81327d66cfd3adbe557aabf26c,d839a45ebe64ab48b60a407d837fb01d3c0dfef9,7182afc4e5c212133d5d7d76eb3df6c24618302b,b9f0208be00bd8b337be7f12e02e3a3ce846e22b,da39a3ee5e6b4b0d3255bfef95601890afd80709,b9f0208be00bd8b337be7f12e02e3a3ce846e22b,b6ffe5619e02871fcd04f61c9bb4b5c53a3f46b7,d839a45ebe64ab48b60a407d837fb01d3c0dfef9,...,2457,5745,2437,2437,14714,15326,046cfa46be49fc10834815c6,5a0546857ecc773753327266,5a0546857ecc773753327266_046cfa46be49fc1083481...,1578474563637
3,da39a3ee5e6b4b0d3255bfef95601890afd80709,da39a3ee5e6b4b0d3255bfef95601890afd80709,b9f0208be00bd8b337be7f12e02e3a3ce846e22b,d839a45ebe64ab48b60a407d837fb01d3c0dfef9,b6ffe5619e02871fcd04f61c9bb4b5c53a3f46b7,b7e6027447eb1f81327d66cfd3adbe557aabf26c,b7e6027447eb1f81327d66cfd3adbe557aabf26c,7182afc4e5c212133d5d7d76eb3df6c24618302b,5731b8e08abc69d4c4d685c58164059207c93310,d839a45ebe64ab48b60a407d837fb01d3c0dfef9,...,5260,2432,2432,5240,19587,18763,046cfa46be49fc10834815c6,5a0546857ecc773753327266,5a0546857ecc773753327266_046cfa46be49fc1083481...,1578474563637
4,da39a3ee5e6b4b0d3255bfef95601890afd80709,da39a3ee5e6b4b0d3255bfef95601890afd80709,7182afc4e5c212133d5d7d76eb3df6c24618302b,b9f0208be00bd8b337be7f12e02e3a3ce846e22b,b6ffe5619e02871fcd04f61c9bb4b5c53a3f46b7,b7e6027447eb1f81327d66cfd3adbe557aabf26c,26571eb257b6a34b6beb93c05768e751d19279a6,d839a45ebe64ab48b60a407d837fb01d3c0dfef9,d839a45ebe64ab48b60a407d837fb01d3c0dfef9,da39a3ee5e6b4b0d3255bfef95601890afd80709,...,5300,5300,5260,5260,22074,22328,046cfa46be49fc10834815c6,5a0546857ecc773753327266,5a0546857ecc773753327266_046cfa46be49fc1083481...,1578474563637
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10101,0f927dce74ec3475c7a39299e5bffab222ca665d,5d998a8668536c4f51004c25f474117fe9555f78,8e69018f6343506344ab13646ccd9447fc2ffb19,efa1fe97c4b2a6c57d98de9c236a109032ceb839,da39a3ee5e6b4b0d3255bfef95601890afd80709,5d998a8668536c4f51004c25f474117fe9555f78,c72257cf75d6a411225948bbc4e37ad554e7beda,da39a3ee5e6b4b0d3255bfef95601890afd80709,5d998a8668536c4f51004c25f474117fe9555f78,da39a3ee5e6b4b0d3255bfef95601890afd80709,...,5200,5200,5280,5785,81664,82589,fd64de8c4a2fc5ebb0e9f412,5dc8cea7659e181adb076a3f,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,1573731060667
10102,8e69018f6343506344ab13646ccd9447fc2ffb19,0f927dce74ec3475c7a39299e5bffab222ca665d,efa1fe97c4b2a6c57d98de9c236a109032ceb839,da39a3ee5e6b4b0d3255bfef95601890afd80709,5d998a8668536c4f51004c25f474117fe9555f78,5d998a8668536c4f51004c25f474117fe9555f78,da39a3ee5e6b4b0d3255bfef95601890afd80709,8e69018f6343506344ab13646ccd9447fc2ffb19,5d998a866853

In [20]:
test_df = test_df[['path_id', 'wp_tmestamp', 'site_id']]
test_df = test_df.rename(columns={'path_id':'path', 'wp_tmestamp':'timestamp', 'site_id':'site'})
test_df

,path,timestamp,site
0,046cfa46be49fc10834815c6,9,5a0546857ecc773753327266
1,046cfa46be49fc10834815c6,9017,5a0546857ecc773753327266
2,046cfa46be49fc10834815c6,15326,5a0546857ecc773753327266
3,046cfa46be49fc10834815c6,18763,5a0546857ecc773753327266
4,046cfa46be49fc10834815c6,22328,5a0546857ecc773753327266
...,...,...,...
10101,fd64de8c4a2fc5ebb0e9f412,82589,5dc8cea7659e181adb076a3f
10102,fd64de8c4a2fc5ebb0e9f412,85759,5dc8cea7659e181adb076a3f
10103,fd64de8c4a2fc5ebb0e9f412,90896,5dc8cea7659e181adb076a3f
10104,fd64de8c4a2fc5ebb0e9f412,96900,5dc8cea7659e181adb076a3f


In [21]:
txt_pathes = []
for path in tqdm(test_df['path'].values):
    txt_pathes.append([test_txt for test_txt in test_txts if path in test_txt][0])
test_df['txt_path'] = txt_pathes

100%|██████████| 10106/10106 [00:00<00:00, 29351.74it/s]


In [22]:
processes = multiprocessing.cpu_count()
with multiprocessing.Pool(processes=processes) as pool:
    dfs = pool.imap_unordered(calc_delta, test_df.groupby('path'))
    dfs = tqdm(dfs)
    dfs = list(dfs)

626it [01:09,  9.02it/s]


In [23]:
test_delta = pd.concat(dfs).sort_index()
test_delta

,timestamp,delta_x,delta_y
0,9,0.000000,0.000000
1,9017,-9.923327,4.431876
2,15326,2.108476,5.745035
3,18763,1.319383,4.042038
4,22328,0.536187,3.816007
...,...,...,...
10101,82589,-7.532983,12.234109
10102,85759,-2.155667,3.983080
10103,90896,-4.763953,7.527394
10104,96900,-8.376678,6.056004


In [24]:
test_delta = test_delta.rename(columns={ 'timestamp':'wp_tmestamp'})

In [25]:
test_df = pd.read_csv(WIFI_DIR / 'test_10_th10000_base25_withpassedtime.csv')

In [26]:
test_df['delta_x'] = test_delta['delta_x']
test_df['delta_y'] = test_delta['delta_y']

In [27]:
test_df.to_csv(DATA_DIR / 'indoorunifiedwifids_original/test_10_th10000_base25_withpassedtime_withdelta.csv', index=False)